This notebook is for testing the application. Read the comments. You can play around with it. 

Make sure to have your api key and your model name configured in `services/config/personal_constants.py`

In [1]:
# importing the class from a different file
from services.querier.querier import DbQuerier

# instantiating the class
test_querier = DbQuerier(sample_db_loc='data/PostNL_SQLite.sqlite')

# do some configurations
test_querier.remove_all_training_data() # if there was any, now there is not
test_querier.train_model(train_on_documentation=True, 
                         train_on_ddl=True, 
                         train_on_sql=False, 
                         train_on_question_sql_pairs=True)

Adding documentation....
Adding documentation....
Adding documentation....
INFO:root:Trained model on documentation files
INFO:root:trained model on sql queries
Adding ddl: CREATE TABLE collo_packages (
	id INTEGER NOT NULL, 
	account_id_hashed VARCHAR, 
	dn_barcode VARCHAR, 
	da_datum_voormelding DATE, 
	da_datum_acceptatie DATE, 
	da_tijd_acceptatie TIME, 
	sa_dag_sortering1 DATE, 
	sa_datum_sortering1 DATE, 
	sa_tijd_sortering1 TIME, 
	sa_datum_distributiecollectie DATE, 
	sa_tijd_distributiecollectie TIME, 
	da_datum_herroutering_voor_up1 DATE, 
	da_tijd_herroutering_voor_up1 TIME, 
	da_datum_eindstatus DATE, 
	da_tijd_eindstatus TIME, 
	ma_gewicht INTEGER, 
	ma_breedte INTEGER, 
	ma_lengte INTEGER, 
	ma_hoogte INTEGER, 
	ma_volume INTEGER, 
	"PC4_gea" INTEGER, 
	da_landcode_gea VARCHAR, 
	da_resultaatgroepcode VARCHAR, 
	da_resultaatcode VARCHAR, 
	da_type_adres_gea VARCHAR, 
	da_waarnemingsequence VARCHAR, 
	PRIMARY KEY (id), 
	FOREIGN KEY(account_id_hashed) REFERENCES delivery_p

In [2]:
# showing the training data that is in the model
test_querier.training_data

,id,training_data_type,question,content
0,211064-sql,sql,Welke maand in 2023 had het laagste aantal pak...,"SELECT month_id, SUM(number_of_parcels) AS tot..."
1,69728-ddl,ddl,None,CREATE TABLE collo_packages (\n\tid INTEGER NO...
2,211066-sql,sql,Wat is het aantal afgeleverde pakketten per ma...,"SELECT month_id, SUM(number_of_parcels) FROM d..."
3,211055-sql,sql,Wat is het gemiddelde aantal pakketten per kla...,SELECT AVG(number_of_parcels) FROM delivery_fa...
4,211063-sql,sql,Hoeveel klanten hebben precies 3 pakketten in ...,SELECT COUNT(*) FROM delivery_facts WHERE numb...
5,211054-sql,sql,Hoeveel pakketten zijn er in totaal geleverd i...,SELECT SUM(number_of_parcels) FROM delivery_fa...
6,211061-sql,sql,Wat is het gemiddelde aantal pakketten per kla...,SELECT AVG(number_of_parcels) FROM delivery_fa...
7,255760-doc,documentation,None,De 'delivery_facts' tabel bevat geaggregeerde ...
8,211056-sql,sql,Hoeveel klanten hebben meer dan 5 pakketten on...,SELECT COUNT(DISTINCT account_id_hashed) FROM ...
9,211065-sql,sql,Hoeveel pakketten zijn in totaal afgeleverd aa...,SELECT SUM(number_of_parcels) FROM delivery_fa...


In [6]:
# some code to play around with
question = """
Welke pakketen liggen klaar om opgehaald te worden door de klant?
"""

sql_code = test_querier.generate_sql(question)
print(sql_code)

SELECT cp.dn_barcode, cp.sa_datum_distributiecollectie, cp.sa_tijd_distributiecollectie 
FROM collo_packages cp 
WHERE cp.da_resultaatgroepcode = '04-Retour';


In [7]:
sample_data = test_querier.generate_sample_data(sql_query=sql_code)
sample_data

,dn_barcode,sa_datum_distributiecollectie,sa_tijd_distributiecollectie
0,33233930000910,2023-08-31,08:12:29.000000
1,3SABAZ7651835,2023-12-13,08:01:49.000000
2,3SABRE7765541,2023-08-05,07:53:11.000000
3,3SABXQ7601437,2023-07-27,08:17:44.000000
4,3SACCU064727101,2023-07-13,09:09:19.000000
...,...,...,...
2856,3SOPKB716291840,2023-11-27,08:37:41.000000
2857,3SOPKC215083710,2023-07-20,09:27:28.000000
2858,3SOPKC230238898,2023-12-01,08:53:03.000000
2859,3SOPKC567733548,2023-09-08,11:25:45.000000
